# Setup Vertex Pipelines

In [ ]:
PROJECT_ID = 'Here is your project ID'
BUCKET_NAME = "Here is your bucket name"
REGION="Region"

PIPELINE_ROOT = f"{BUCKET_NAME}/POC/"
print(PIPELINE_ROOT)

# Define the pipeline

In [2]:
from kfp import components
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath
from google.oauth2 import service_account
from google.cloud import aiplatform
from datetime import datetime

In [3]:
concat_operation = components.load_component_from_file('config/concat_comp.yaml')
help(concat_operation)

Help on function Concat strings:

Concat strings(parameter_1: str, parameter_2: str = '100')
    Concat strings
    Concat 2 strings and save the result



In [4]:
reverse_operation = components.load_component_from_file('config/reverse_comp.yaml')
help(reverse_operation)

Help on function Reverse:

Reverse(input_1: 'Dataset')
    Reverse
    Reverse a string from a file



In [8]:
metrics_operation = components.load_component_from_file('config/metrics.yaml')
help(metrics_operation)

Help on function Yaml Component:

Yaml Component(hola: 'Dataset', param1: str)
    Yaml Component
    Yaml Component Test



## Build the Pipeline Function

In [6]:
@pipeline(
    # A name and description for the pipeline.
    name='poc-pipeline',
    description='A pipeline which concatenates and reverse strings and shows metrics',
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT
)
def pipeline(
    str1: str = 'Hello',
    str2: str = 'World',
    param1: str ='0.5',
):
    concat_task = concat_operation(str1,str2)

    model_task = reverse_operation(
        concat_task.output
    )
    metrics_task = metrics_operation(model_task.output,param1)

## Compile the Pipeline

In [7]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="config/POC_pipeline.json"
)

D:\Documents\PycharmProjects\ChiperEnv\lib\site-packages\kfp\v2\compiler\compiler.py:1278: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


## Create a Vertex AI Pipelines Experiment

In [8]:
credentials = service_account.Credentials.from_service_account_file("path_to_service_account_file")

In [9]:
aiplatform.init(project=PROJECT_ID, credentials=credentials)

In [40]:
TIMESTAMP = datetime.utcnow().strftime("%Y-%m-%d-utc-%H-%M-%S")

In [43]:
run1 = aiplatform.PipelineJob(
    display_name="poc-pipeline",
    template_path="config/POC_pipeline.json",
    job_id="poc-pipeline-{0}".format(TIMESTAMP),
    labels={"pipeline-project-id": "poc-pipeline", "environment": "develop"},
    enable_caching=True
)

In [44]:
run1.submit()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/645165474319/locations/us-central1/pipelineJobs/poc-pipeline-2022-04-26-utc-18-14-19
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/645165474319/locations/us-central1/pipelineJobs/poc-pipeline-2022-04-26-utc-18-14-19')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/poc-pipeline-2022-04-26-utc-18-14-19?project=645165474319
